In [15]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import pickle as pk
import base64
from PIL import Image
import io
from contextlib import asynccontextmanager

In [16]:
import uvicorn
import nest_asyncio
nest_asyncio.apply()

In [17]:
# data classes
class PredictionResponse(BaseModel):
    prediction: float

class ImageRequest(BaseModel):
    image: str

In [21]:
# lifecicle - app

@asynccontextmanager
async def lifespan(app: FastAPI):
    global xgb_model
    with open('../modelos/model_tree.pkl', 'rb') as f:
        xgb_model = pk.load(f)

    yield; del xgb_model

app = FastAPI(lifespan=lifespan)

In [22]:
#predict
@app.post('/predict', response_model=PredictionResponse)
async def predict(request: ImageRequest):
    img_bytes = base64.b64decode(request.image)
    img = Image.open(io.BytesIO(img_bytes)).resize((28, 28))
    img_array = np.array(img)
    img_array_gs = np.dot(img_array[..., :3], [.2989, .5870, .1140]).reshape(1, -1)

    return {'prediction': xgb_model.predict(img_array_gs)}

In [23]:
#healthcheck
@app.get('/healthcheck')
async def healthcheck():
    return {'status': 'alive'}

In [24]:
uvicorn.run(app, host='localhost', port=8000)

INFO:     Started server process [81279]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [81279]
